In [2]:
#Update your token
STUDENT_TOKEN = 'TOMAS-MESALLES'

Server communication game and game_loop.

In [3]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    print(self.state)
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player=False, id=None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mwinner\033[0m' if game.player == player else '\033[91mloser\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

Random solver

In [4]:
def random_solver(game):
  actions = game.actions()
  action = random.choice(actions)
  return action

Game specific code

In [5]:
from functools import reduce
from copy import copy, deepcopy
import json

class NineMensMorris(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return self.state['actions']

  def print_game(self):
    print('Board: ' + ' '.join(self.state['board']))

  def other_player(self):
    if self.player == 'X': return 'O'
    if self.player == 'O': return 'X'

In [6]:
game_loop(random_solver, NineMensMorris, 'ninemensmorris', multi_player=False, id=None)

Creating new game...
Joining game with id: 17726
Playing as O
Waiting for the other player to join...
Board: . . . . . X . . . . . . . . . . . . . . . . . .
Making next move...
b'Valid move'
Board: . . . O . X . . . . . . . . . . . . . . . . . X
Making next move...
b'Valid move'
Board: . . . O . X . . O . . . . . . . . . . . X . . X
Making next move...
b'Valid move'
Board: . . . O . X . . O . . . O . . . . . X . X . . X
Making next move...
b'Valid move'
Board: . . . . . X . . O . . O O . . . . . X X X . . X
Making next move...
b'Valid move'
Board: . . . . O X . . O . . O O . . . . X X X X . . X
Making next move...
b'Valid move'
Board: . . . . . X . . O . . O O O . . X X X X X . . X
Making next move...
b'Valid move'
Board: O . . . . X . . O . . . O O . . X X X X X . X X
Making next move...
b'Valid move'
Board: O . X . . X . . O . . . O O . O X X X X X . X X
Making next move...
b'Valid move'
Board: O . X . . X . O O . . . O O . O . X X X X . X X
Making next move...
b'Valid move'
Board: O